In [10]:
#Resnet18
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

# Define ResNet-18 architecture
def resnet_block(input_tensor, filters, strides=(1, 1), downsample=False):
    identity = input_tensor
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample:
        identity = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(identity)
        identity = layers.BatchNormalization()(identity)
    x = layers.add([x, identity])
    x = layers.Activation('relu')(x)
    return x

def resnet18(input_shape=(32, 32, 3), num_classes=100):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = resnet_block(x, 64)
    x = resnet_block(x, 64)
    x = resnet_block(x, 128, downsample=True)
    x = resnet_block(x, 128)
    x = resnet_block(x, 256, downsample=True)
    x = resnet_block(x, 256)
    x = resnet_block(x, 512, downsample=True)
    x = resnet_block(x, 512)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# Create ResNet-18 model
model = resnet18()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')



Epoch 1/20
782/782 [==============================] - 54s 53ms/step - loss: 3.6137 - accuracy: 0.1497 - val_loss: 4.2021 - val_accuracy: 0.1222
Epoch 2/20
782/782 [==============================] - 38s 49ms/step - loss: 2.8554 - accuracy: 0.2751 - val_loss: 3.5164 - val_accuracy: 0.2158
Epoch 3/20
782/782 [==============================] - 38s 49ms/step - loss: 2.4491 - accuracy: 0.3561 - val_loss: 2.9016 - val_accuracy: 0.2932
Epoch 4/20
782/782 [==============================] - 38s 49ms/step - loss: 2.1587 - accuracy: 0.4196 - val_loss: 2.4881 - val_accuracy: 0.3663
Epoch 5/20
782/782 [==============================] - 38s 49ms/step - loss: 1.9264 - accuracy: 0.4728 - val_loss: 2.3023 - val_accuracy: 0.4047
Epoch 6/20
782/782 [==============================] - 39s 49ms/step - loss: 1.7319 - accuracy: 0.5207 - val_loss: 2.7916 - val_accuracy: 0.3390
Epoch 7/20
782/782 [==============================] - 39s 49ms/step - loss: 1.5540 - accuracy: 0.5614 - val_loss: 2.4463 - val_accuracy:

In [9]:
#Resnet 18 with Squeeze and excitation attention
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

# Define SE block
def se_block(input_tensor, ratio=16):
    num_channels = input_tensor.shape[-1]
    squeeze = layers.GlobalAveragePooling2D()(input_tensor)
    excitation = layers.Dense(num_channels // ratio, activation='relu')(squeeze)
    excitation = layers.Dense(num_channels, activation='sigmoid')(excitation)
    excitation = tf.expand_dims(tf.expand_dims(excitation, axis=1), axis=1)
    return layers.multiply([input_tensor, excitation])

# Define ResNet-18 with SE blocks
def resnet18_se(input_shape=(32, 32, 3), num_classes=100):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(512, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.Conv2D(512, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# Create ResNet-18 with SE blocks
model = resnet18_se()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/20
782/782 [==============================] - 39s 35ms/step - loss: 3.3821 - accuracy: 0.1746 - val_loss: 3.1741 - val_accuracy: 0.2200
Epoch 2/20
782/782 [==============================] - 25s 32ms/step - loss: 2.3898 - accuracy: 0.3611 - val_loss: 3.4089 - val_accuracy: 0.2560
Epoch 3/20
782/782 [==============================] - 25s 32ms/step - loss: 1.9084 - accuracy: 0.4728 - val_loss: 2.4713 - val_accuracy: 0.3745
Epoch 4/20
782/782 [==============================] - 25s 32ms/step - loss: 1.5814 - accuracy: 0.5506 - val_loss: 2.2160 - val_accuracy: 0.4283
Epoch 5/20
782/782 [==============================] - 26s 34ms/step - loss: 1.3214 - accuracy: 0.6178 - val_loss: 2.4902 - val_accuracy: 0.3952
Epoch 6/20
782/782 [==============================] - 26s 34ms/step - loss: 1.0835 - accuracy: 0.6797 - val_loss: 2.1956 - val_accuracy: 0.4640
Epoch 7/20
782/782 [==============================] - 25s 32ms/step - loss: 0.8646 - accuracy: 0.7372 - val_loss: 1.9459 - val_accuracy:

In [2]:
#Simple CNN
import numpy as np
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

# Create CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
epochs = 10
history = model.fit(x_train, y_train, epochs=epochs, batch_size=64, validation_split=0.2)

# Display training accuracy and loss for each epoch
for epoch in range(epochs):
    print("Epoch", epoch+1, "Training Accuracy:", history.history['accuracy'][epoch], "Training Loss:", history.history['loss'][epoch])

# Evaluate model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)


169001437/169001437 [==============================] - 2s 0us/step
Epoch 1/10
625/625 [==============================] - 10s 8ms/step - loss: 4.3839 - accuracy: 0.0303 - val_loss: 4.0911 - val_accuracy: 0.0743
Epoch 2/10
625/625 [==============================] - 5s 7ms/step - loss: 3.9905 - accuracy: 0.0803 - val_loss: 3.7248 - val_accuracy: 0.1222
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 3.7797 - accuracy: 0.1101 - val_loss: 3.5066 - val_accuracy: 0.1656
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 3.6422 - accuracy: 0.1292 - val_loss: 3.3911 - val_accuracy: 0.1874
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 3.5423 - accuracy: 0.1500 - val_loss: 3.3436 - val_accuracy: 0.2033
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 3.4530 - accuracy: 0.1639 - val_loss: 3.2043 - val_accuracy: 0.2199
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 3.3884

In [8]:
#Simple CNN with squeeze and excitation attention
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

# Define SE block
def se_block(input_tensor, ratio=16):
    num_channels = input_tensor.shape[-1]
    squeeze = layers.GlobalAveragePooling2D()(input_tensor)
    excitation = layers.Dense(num_channels // ratio, activation='relu')(squeeze)
    excitation = layers.Dense(num_channels, activation='sigmoid')(excitation)
    excitation = tf.expand_dims(tf.expand_dims(excitation, axis=1), axis=1)
    return layers.multiply([input_tensor, excitation])

# Define CNN model with SE block
def simple_cnn_se():
    inputs = layers.Input(shape=(32, 32, 3))
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = se_block(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = se_block(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = se_block(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(100, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

# Create the model
model = simple_cnn_se()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/10
782/782 [==============================] - 14s 11ms/step - loss: 3.8091 - accuracy: 0.1179 - val_loss: 3.2456 - val_accuracy: 0.2160
Epoch 2/10
782/782 [==============================] - 8s 11ms/step - loss: 2.9765 - accuracy: 0.2639 - val_loss: 2.7926 - val_accuracy: 0.3036
Epoch 3/10
782/782 [==============================] - 8s 11ms/step - loss: 2.5515 - accuracy: 0.3476 - val_loss: 2.5654 - val_accuracy: 0.3547
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 2.2290 - accuracy: 0.4196 - val_loss: 2.4209 - val_accuracy: 0.3867
Epoch 5/10
782/782 [==============================] - 9s 11ms/step - loss: 1.9587 - accuracy: 0.4778 - val_loss: 2.3149 - val_accuracy: 0.4120
Epoch 6/10
782/782 [==============================] - 9s 11ms/step - loss: 1.7111 - accuracy: 0.5360 - val_loss: 2.4431 - val_accuracy: 0.3967
Epoch 7/10
782/782 [==============================] - 9s 12ms/step - loss: 1.4741 - accuracy: 0.5903 - val_loss: 2.4077 - val_accuracy: 0.416

In [11]:
#Mobilenet
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

# Define MobileNet architecture
def mobile_net(input_shape=(32, 32, 3), num_classes=100):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = depthwise_separable_conv_block(x, 64, (3, 3))
    x = depthwise_separable_conv_block(x, 128, (3, 3), strides=(2, 2))
    x = depthwise_separable_conv_block(x, 128, (3, 3))

    x = depthwise_separable_conv_block(x, 256, (3, 3), strides=(2, 2))
    x = depthwise_separable_conv_block(x, 256, (3, 3))

    x = depthwise_separable_conv_block(x, 512, (3, 3), strides=(2, 2))
    for _ in range(5):
        x = depthwise_separable_conv_block(x, 512, (3, 3))

    x = depthwise_separable_conv_block(x, 1024, (3, 3), strides=(2, 2))
    x = depthwise_separable_conv_block(x, 1024, (3, 3))

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

def depthwise_separable_conv_block(input_tensor, filters, kernel_size, strides=(1, 1)):
    x = layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, (1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    return x

# Create MobileNet model
model = mobile_net()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/20
782/782 [==============================] - 37s 26ms/step - loss: 4.2698 - accuracy: 0.0586 - val_loss: 3.9954 - val_accuracy: 0.0836
Epoch 2/20
782/782 [==============================] - 19s 24ms/step - loss: 3.7628 - accuracy: 0.1251 - val_loss: 4.1732 - val_accuracy: 0.1157
Epoch 3/20
782/782 [==============================] - 18s 24ms/step - loss: 3.5268 - accuracy: 0.1572 - val_loss: 5.2598 - val_accuracy: 0.1259
Epoch 4/20
782/782 [==============================] - 18s 23ms/step - loss: 3.2976 - accuracy: 0.1941 - val_loss: 3.6979 - val_accuracy: 0.1686
Epoch 5/20
782/782 [==============================] - 19s 24ms/step - loss: 3.1198 - accuracy: 0.2217 - val_loss: 4.0689 - val_accuracy: 0.1579
Epoch 6/20
782/782 [==============================] - 18s 23ms/step - loss: 2.9576 - accuracy: 0.2535 - val_loss: 3.1841 - val_accuracy: 0.2275
Epoch 7/20
782/782 [==============================] - 19s 24ms/step - loss: 2.8087 - accuracy: 0.2784 - val_loss: 3.1313 - val_accuracy:

In [12]:
# Mobilenet with Squeeze and excitation attention

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
# Load CIFAR-100 data
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert class vectors to binary class matrices (for use with categorical_crossentropy)
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

def squeeze_excite_block(input_tensor, ratio=16):
    init = input_tensor
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = layers.GlobalAveragePooling2D()(init)
    se = layers.Reshape(se_shape)(se)
    se = layers.Dense(filters // ratio, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)

    x = layers.multiply([init, se])
    return x
def depthwise_separable_conv_block(input_tensor, filters, kernel_size, strides=(1, 1), include_se=False):
    x = layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, (1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    if include_se:
        x = squeeze_excite_block(x)

    return x

def MobileNetSE(input_shape=(32, 32, 3), num_classes=100):
    inputs = layers.Input(shape=input_shape)

    # Adapted MobileNet architecture with SE blocks
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = depthwise_separable_conv_block(x, 64, (3, 3), include_se=True)
    x = depthwise_separable_conv_block(x, 128, (3, 3), strides=(2, 2), include_se=True)
    x = depthwise_separable_conv_block(x, 128, (3, 3), include_se=True)
    x = depthwise_separable_conv_block(x, 256, (3, 3), strides=(2, 2), include_se=True)
    x = depthwise_separable_conv_block(x, 256, (3, 3), include_se=True)
    x = depthwise_separable_conv_block(x, 512, (3, 3), strides=(2, 2), include_se=True)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Instantiate and compile the model
model = MobileNetSE()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/20
782/782 [==============================] - 29s 23ms/step - loss: 3.5903 - accuracy: 0.1570 - val_loss: 3.2334 - val_accuracy: 0.2125
Epoch 2/20
782/782 [==============================] - 17s 22ms/step - loss: 2.8632 - accuracy: 0.2802 - val_loss: 2.9675 - val_accuracy: 0.2768
Epoch 3/20
782/782 [==============================] - 17s 21ms/step - loss: 2.5077 - accuracy: 0.3511 - val_loss: 2.7064 - val_accuracy: 0.3156
Epoch 4/20
782/782 [==============================] - 17s 22ms/step - loss: 2.2471 - accuracy: 0.4074 - val_loss: 2.6051 - val_accuracy: 0.3504
Epoch 5/20
782/782 [==============================] - 17s 22ms/step - loss: 2.0382 - accuracy: 0.4493 - val_loss: 2.4109 - val_accuracy: 0.3845
Epoch 6/20
782/782 [==============================] - 17s 21ms/step - loss: 1.8633 - accuracy: 0.4925 - val_loss: 2.5199 - val_accuracy: 0.3753
Epoch 7/20
782/782 [==============================] - 17s 22ms/step - loss: 1.7092 - accuracy: 0.5282 - val_loss: 2.3728 - val_accuracy: